# サムネイルのURLを取得

In [1]:
from apiclient.discovery import build

DEVELOPER_KEY = '****'

MAIN = 'UCutJqz56653xV2wwSvut_hQ'  # メインチャンネルのID
SUB = 'UCynIYcsBwTrwBIecconPN2A'  # サブチャンネルのID

TERM = [['2016-01-01T00:00:01Z', '2017-01-01T00:00:00Z'],
        ['2017-01-01T00:00:01Z', '2018-01-01T00:00:00Z'],
        ['2018-01-01T00:00:01Z', '2019-01-01T00:00:00Z'],
        ['2019-01-01T00:00:01Z', '2020-01-01T00:00:00Z']]

MAIN_FILE = './main_thumbnail.txt'
SUB_FILE = './sub_thumbnail.txt'

main_videos = []
sub_videos = []

## 動画のvideo IDを取得

In [2]:
def get_videoId(channelId, pageToken, videos, t):
    youtube = build('youtube', 'v3', developerKey=DEVELOPER_KEY)

    search_response = youtube.search().list(
        part='snippet',
        maxResults=50,
        channelId=channelId,
        order='date',
        pageToken=pageToken,
        publishedAfter=t[0],
        publishedBefore=t[1]
    ).execute()
    
    for search_result in search_response.get('items', []):
        if search_result['id']['kind'] == 'youtube#video':
            videos.append(search_result['id']['videoId'])

    try:
        nextPageToken = search_response['nextPageToken']
        get_videoId(channelId, nextPageToken, videos, t)
    except:
        return

## サムネイルのURLを取得

In [3]:
def get_url(videos, file):
    youtube = build('youtube', 'v3', developerKey=DEVELOPER_KEY)
    
    with open(file, mode='w') as f:
        for videoId in videos:
            search_response = youtube.videos().list(
                part='snippet',
                id=videoId
            ).execute()
        
            search_result = search_response.get('items', [])
            f.write(search_result[0]['snippet']['thumbnails']['default']['url'] + '\n')

In [4]:
for t in TERM:
    get_videoId(MAIN, '', main_videos, t)
get_url(main_videos, MAIN_FILE)

for t in TERM:
    get_videoId(SUB, '', sub_videos, t)
get_url(sub_videos, SUB_FILE)